In [2]:
import os
import cv2
import numpy as np
from natsort import natsorted
import torch
import matplotlib.pyplot as plt
from PIL import Image
from multiprocessing import Pool
from sklearn.preprocessing import MinMaxScaler
from skimage import exposure
import math
import shutil

In [6]:
def toTexture(image, gt_image, count):
    
    background = cv2.imread(gt_image)
    groundTruth = background.copy()
    texture_img = cv2.imread(image)
    
    texture_img_gray = cv2.cvtColor(texture_img, cv2.COLOR_BGR2GRAY)
    th, texture_th = cv2.threshold(texture_img_gray, 30, 50, cv2.THRESH_BINARY_INV)
    
    texture_th_floodfill = texture_th.copy()
    
    mask = np.zeros((image_height + 2, image_width + 2), np.uint8)
    
    cv2.floodFill(texture_th_floodfill, mask, (0,0), 255)
    
    texture_th_floodfill_inv = cv2.bitwise_not(texture_th_floodfill)
    
    im_out = texture_th | texture_th_floodfill_inv
    
    index_texture = np.array(list(zip(*np.where(im_out == 255))))
    
    background[index_texture[:,0], index_texture[:,1]] = texture_img[index_texture[:,0], index_texture[:,1]]
    
    texture_result = cv2.seamlessClone(background, groundTruth, im_out, (image_height//2, image_width // 2), cv2.NORMAL_CLONE)
    
    cv2.imwrite(os.path.join(src_directory, 'merged_texture', '{}.jpg'.format(count)), texture_result)
    

In [11]:
image_height = 256
image_width = 256
texture_height = 280
texture_width = 280
src_directory = '../audioDVP_files/'

In [13]:
texture_image_files = natsorted([os.path.join(src_directory,'predicted_texture', x) for x in os.listdir(os.path.join(src_directory, 'predicted_texture'))])
gt_frames = natsorted([os.path.join(src_directory, 'crop', x) for x in os.listdir(os.path.join(src_directory, 'crop'))])
# reference_mouth = Image.fromarray(reference_mouth)
count = [i for i in range(len(texture_image_files))]

In [17]:
if os.path.exists(os.path.join(src_directory, 'merged_texture')):
    shutil.rmtree(os.path.join(src_directory, 'merged_texture'))
os.makedirs(os.path.join(src_directory, 'merged_texture'))
pool = Pool(processes=40)
pool.starmap(toTexture, zip(texture_image_files, gt_frames, count))
pool.terminate()
pool.join()

In [24]:
# os.system('ffmpeg -y -i {}/%d.jpg -i {} -c:v libopenh264 -r 25 {}'.format(os.path.join(src_directory, 'predicted_face'), os.path.join(src_directory, 'audio','audio.wav'), os.path.join(src_directory, 'results', 'predicted_face.mp4')))
os.system('ffmpeg -y -i {}/%d.jpg -c:v libx264 -crf 1 -r 25 {}'.format(os.path.join(src_directory, 'merged_texture'), os.path.join(src_directory, 'results', 'final_outcome.mp4')))
os.system('ffmpeg -y -i {} -i {} -c:v copy -c:a copy {}'.format(os.path.join(src_directory, 'results', 'final_outcome.mp4'), os.path.join(src_directory,'reenact_audio2bfm_1e_2.mp4'), os.path.join(src_directory, 'results', 'with_audio.mp4')))

0

In [66]:
os.system('ffmpeg -y -i {}/%d.jpg -i {} -c:v libx264 -crf 1 -r 25 {}'.format(os.path.join(src_directory, 'merged_texture'), os.path.join(src_directory, 'audio','audio.wav'), os.path.join(src_directory, 'results', 'final_texture.mp4')))

0

In [47]:
out = cv2.VideoWriter(os.path.join(src_directory, 'temp_mesh.mp4'), cv2.VideoWriter_fourcc(*'mp4v'), 25, (image_width, image_height))

imageFiles = natsorted([os.path.join(src_directory, 'reenact_mesh_image', x) for x in os.listdir(os.path.join(src_directory, 'reenact_mesh_image'))])

for im in imageFiles:
    image = cv2.imread(im)
    out.write(image)

out.release()
os.system('ffmpeg -y -i {} -i {} -c:v copy -c:a aac {}'.format(os.path.join(src_directory, 'temp_mesh.mp4')
, os.path.join(src_directory, 'audio','audio.wav'), os.path.join(src_directory, 'results', 'predicted_mesh.mp4')
))

0

In [17]:
os.system('ffmpeg -y -i {}/results/predicted_face.mp4 -i {}/results/predicted_mesh.mp4 -filter_complex hstack -c:v libopenh264 {}/results/face_mesh_comparison.mp4'.format(src_directory,src_directory,src_directory))

0